In [1]:
#import du fichier sgf
from sgf_to_states import sgf_to_states
nom_fichier="Mi_Yuting-Park_Junghwan.sgf"
taille_plateau=19

liste = sgf_to_states(nom_fichier,taille_plateau)

In [50]:
#visualisation d'un état du jeu
from go import View
# il faut utiliser :  print( View(liste[i]))


In [51]:
#visualisation d'un layer
def vis_layer(layer,num_layer):
    x=0
    taille=len(layer[num_layer,:,:])
    while x < taille:
        string =""
        y=0
        while y < taille:
            if layer[num_layer,y,x]==1:  # il faut inverser x et y ici
                string = string + " 1"    
            else:
                string = string + " ."
            y+=1
        print string
        x+=1

![title](img/Capture.JPG)

In [52]:
import numpy as np
print liste[0]._width

19


In [53]:
def get_stonecolour(state):
    taille=state._width
    planes = np.zeros((3, taille,taille))
    x=0
    while x<taille:
        y=0
        while y<taille:
            case=repr(state[x+1,y+1]) # conversion en string sinon le test == ne marche pas
            if case=='Black': # pion noir
                planes[0, x, y] = 1 
            if case=='White': #pion blanc
                planes[1, x, y] = 1 
            if case=='Empty': #vide
                planes[2, x, y] = 1 
            y+=1
        x+=1
    
    return planes


In [54]:
#stone colour
tenseur_coup60=get_stonecolour(liste[60])
print View(liste[60])
print 
vis_layer(tenseur_coup60,0)
print 
vis_layer(tenseur_coup60,1)
print 
vis_layer(tenseur_coup60,2)

X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. . . o . . . * . * o * * o o . o . .
. . o o * . . . * o o o . . . * o . .
. . * . o . . . o . . . . . o o * * .
. * . * o . . . . . . . . . . * o * .
. . o * o . . . . . . . . . . . . . .
. * * o . . . . . . . . . . . . * . .
. * o o . . . . . . . . . . . . . . .
. . * * . . . . . + . . . . . + o . .
. o . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . o . .
. . o . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . o o . . . . . + . . . . . + * . .
. . * . . . . . . . . . . * . . . . .
. * . * . o . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .

 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . 1 1 1 . . 1 . . . . .
 . . . . . . . 1 . 1 . 1 1 . . . . . .
 . . . . 1 . . . 1 . . . . . . 1 . . .
 . . 1 . . . . . . . . . . . . . 1 1 .
 . 1 . 1 . . . . . . . . . . . 1 . 1 .
 . . . 1 . . . . . . . . . . . . . . .
 . 1

In [55]:
#ones
ones=np.ones((1, taille,taille))
# ajout de ones au premier tenseur
tenseur_coup60=np.concatenate([tenseur_coup60,ones], axis=0)
tenseur_coup60.shape

(4L, 19L, 19L)

In [85]:
#problème au bord on ne peut pas appeler count_liberties sur les bords du plateau...
def get_liberties(state):
    taille=state._width
    planes = np.zeros((8, taille,taille))
    for x,y in state._legal:
        n_liberties=state.count_liberties(x,y)
        if n_liberties>8 :
            n_liberties = 8
        if not(n_liberties==0):   #pk ?
            planes[n_liberties-1,x,y]=1
    return planes

In [86]:
liberties=get_liberties(liste[60])
print View(liste[60])
print 
vis_layer(liberties,0)
print 
vis_layer(liberties,1)
print 
vis_layer(liberties,2)

X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. . . o . . . * . * o * * o o . o . .
. . o o * . . . * o o o . . . * o . .
. . * . o . . . o . . . . . o o * * .
. * . * o . . . . . . . . . . * o * .
. . o * o . . . . . . . . . . . . . .
. * * o . . . . . . . . . . . . * . .
. * o o . . . . . . . . . . . . . . .
. . * * . . . . . + . . . . . + o . .
. o . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . o . .
. . o . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . o o . . . . . + . . . . . + * . .
. . * . . . . . . . . . . * . . . . .
. * . * . o . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .

 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . 1 . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . 1 . .
 . . . . . . . . . . . . . . . . . . .
 . .

In [58]:
# ajout de liberties au 1er tenseur
tenseur_coup60=np.concatenate([tenseur_coup60,liberties], axis=0)
tenseur_coup60.shape

(12L, 19L, 19L)

In [59]:
#/!\ fonction qui ne donne pas exactement l'historique de la partie masi le dernier changement de couleur d'une case /!\
def get_history(state):
    taille=state._width
    planes = np.zeros((8, taille,taille))
    for x,y in state._legal:
        i=1
        case=repr(state[x,y])                           # jmet un +1 psk tu as mis un +1
        while i <8 : 
            hist = state._history[-i][0]
            case_hist = repr(hist[x][y])
            if (not(case==hist)and not(case=="Empty")): # on teste si la case a changé et i sert à savoir quand
                planes[i-1,x,y]=1
                i=9                                     # on retient que le dernier changement de couleur d'une case donc on passse à la suivante
            i=i+1
        if (not(case=="Empty") and (i==8)):             # toutes les cases non vides qu'on a pas prises sont >8
            planes[7,x,y]=1
    return planes

In [61]:
history=get_history(liste[60])
print View(liste[60])
print 
vis_layer(history,0)
print 
vis_layer(history,1)
print 
vis_layer(history,2)

X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. . . o . . . * . * o * * o o . o . .
. . o o * . . . * o o o . . . * o . .
. . * . o . . . o . . . . . o o * * .
. * . * o . . . . . . . . . . * o * .
. . o * o . . . . . . . . . . . . . .
. * * o . . . . . . . . . . . . * . .
. * o o . . . . . . . . . . . . . . .
. . * * . . . . . + . . . . . + o . .
. o . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . o . .
. . o . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . o o . . . . . + . . . . . + * . .
. . * . . . . . . . . . . * . . . . .
. * . * . o . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .

 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . 1 1 1 . . 1 . . . .
 . . . . 1 . . . 1 . 1 1 1 1 1 1 . 1 .
 . . . 1 1 1 . . . 1 1 1 1 . . . 1 1 .
 . . . 1 . 1 . . . 1 . . . . . 1 1 1 1
 . . 1 . 1 1 . . . . . . . . . . 1 1 1
 . .

In [62]:
# ajout de history au 1er tenseur
tenseur_coup60=np.concatenate([tenseur_coup60,history], axis=0)
tenseur_coup60.shape

(20L, 19L, 19L)

In [63]:
def get_captured(state):
    taille=state._width
    planes = np.zeros((8, taille,taille))
    for x,y in state._legal:
            state_copy = state
            n_captured = state_copy._take_pieces(x, y)
            if n_captured>8 :
                n_captured = 8
            if not(n_captured==0):
                planes[n_captured-1,x,y]=1

    return planes

In [65]:
captured=get_captured(liste[60])
print View(liste[60])
print 
vis_layer(captured,0)
print 
vis_layer(captured,1)
print 
vis_layer(captured,2)

X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. . . o . . . * . * o * * o o . o . .
. . o o * . . . * o o o . . . * o . .
. . * . o . . . o . . . . . o o * * .
. * . * o . . . . . . . . . . * o * .
. . o * o . . . . . . . . . . . . . .
. * * o . . . . . . . . . . . . * . .
. * o o . . . . . . . . . . . . . . .
. . * * . . . . . + . . . . . + o . .
. o . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . o . .
. . o . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . o o . . . . . + . . . . . + * . .
. . * . . . . . . . . . . * . . . . .
. * . * . o . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .

 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . .

In [66]:
# ajout de captured au 1er tenseur
tenseur_coup60=np.concatenate([tenseur_coup60,captured], axis=0)
tenseur_coup60.shape

(28L, 19L, 19L)

In [71]:
def self_atari(state):
    taille=state._width
    planes = np.zeros((8, taille,taille))
    
    
    
    return planes


In [87]:
def get_liberties_after(state_after):
    planes = get_liberties(state_after)
    return planes


In [88]:
lib_after=get_liberties_after(liste[61])
print View(liste[60])
print 
vis_layer(lib_after,0)
print 
vis_layer(lib_after,1)
print 
vis_layer(lib_after,2)

X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. . . o . . . * . * o * * o o . o . .
. . o o * . . . * o o o . . . * o . .
. . * . o . . . o . . . . . o o * * .
. * . * o . . . . . . . . . . * o * .
. . o * o . . . . . . . . . . . . . .
. * * o . . . . . . . . . . . . * . .
. * o o . . . . . . . . . . . . . . .
. . * * . . . . . + . . . . . + o . .
. o . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . o . .
. . o . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . o o . . . . . + . . . . . + * . .
. . * . . . . . . . . . . * . . . . .
. * . * . o . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .

 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . 1 . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . 1 . .
 . . . . . . . . . . . . . . . . . . .
 . .

In [ ]:
def ladder_capture(state):
    taille=state._width
    planes = np.zeros((1, taille,taille))
    
    
    return planes


In [ ]:
def ladder_escape(state):
    taille=state._width
    planes = np.zeros((1, taille,taille))
    
    
    return planes


In [83]:
def get_sensibleness(state):
    taille=state._width
    planes = np.ones((1, taille,taille))
    for x,y in state._legal:
        planes[0,x-1,y-1]=0
    return planes


In [84]:
sensibleness=get_sensibleness(liste[60])
print View(liste[60])
print 
vis_layer(sensibleness,0)

X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. . . o . . . * . * o * * o o . o . .
. . o o * . . . * o o o . . . * o . .
. . * . o . . . o . . . . . o o * * .
. * . * o . . . . . . . . . . * o * .
. . o * o . . . . . . . . . . . . . .
. * * o . . . . . . . . . . . . * . .
. * o o . . . . . . . . . . . . . . .
. . * * . . . . . + . . . . . + o . .
. o . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . o . .
. . o . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . o o . . . . . + . . . . . + * . .
. . * . . . . . . . . . . * . . . . .
. * . * . o . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .

 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 . . . 1 1 . 1 1 1 1 1
 1 1 1 . 1 1 1 . . . . . . . . 1 . 1 1
 1 1 . . . 1 1 1 . . . . 1 1 1 . . 1 1
 1 1 . 1 . 1 1 1 . 1 1 1 1 1 . . . . 1
 1 . 1 . . 1 1 1 1 1 1 1 1 1 1 . . . 1
 1 1 . . . 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 .

In [68]:
#zero
zero=np.zeros((1, taille,taille))
# ajout de zero au premier tenseur
#tenseur_coup60=np.concatenate([tenseur_coup60,zero], axis=0) # a decommenter en dernier
tenseur_coup60.shape

(28L, 19L, 19L)